In [126]:
import pandas as pd
import nltk
from nltk import pos_tag, word_tokenize
from nltk.tokenize import sent_tokenize
from nltk.corpus import stopwords
nltk.download('stopwords')
nltk.download('punkt')
 
from tqdm import tqdm
import re
import string
from itertools import combinations
from collections import Counter


from flair.models import SequenceTagger
from flair.data import Sentence

from fuzzywuzzy import fuzz
from collections import defaultdict
from fuzzywuzzy import process

[nltk_data] Downloading package stopwords to /home/julian/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /home/julian/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [293]:
# Importing movie and character metadata
data_folder = './Data/'
character_metadata = pd.read_csv(data_folder+'characters_data.csv')
movie_metadata = pd.read_csv(data_folder+'movie_data.csv')


def select_elligible_movies_for_main_char_analysis(movie_metadata):
    # Removing movies without summaries
    print(len(movie_metadata))
    elligible_movies = movie_metadata[movie_metadata['plot_summary'].notna()]
    print(len(elligible_movies))
    
    # Removing movies with no listed characters
    movies_with_no_listed_char_wiki_ID = [id for id in elligible_movies['wikipedia_ID'] if character_metadata.loc[character_metadata['wikipedia_ID']==id].empty]
    movies_with_no_listed_char_IMDB_ID = [id for id in elligible_movies['IMDB_ID'] if character_metadata.loc[character_metadata['IMDB_ID']==id].empty]
    movies_with_no_listed_char = list(set(movies_with_no_listed_char_wiki_ID).intersection(movies_with_no_listed_char_IMDB_ID))
    elligible_movies = elligible_movies[~elligible_movies[id_type].isin(movies_with_no_listed_char)]
    print(len(elligible_movies))

    id_types = ['wikipedia_ID','IMDB_ID']
    for id_type in id_types:
        print("loop")

        # Removing movies with characters having name listed as nan
        movies_with_nan_characters_ID = pd.Series(character_metadata[character_metadata['character_name'].isna()][id_type].unique())
        movies_with_nan_characters_ID = movies_with_nan_characters_ID.dropna().tolist()
        elligible_movies = elligible_movies[~elligible_movies[id_type].isin(movies_with_nan_characters_ID)]
        print(len(elligible_movies))

        # Removing movies with characters having gender listed as nan
        movies_with_nan_genders_ID = pd.Series(character_metadata[character_metadata['actor_gender'].isna()][id_type].unique())
        movies_with_nan_genders_ID = movies_with_nan_genders_ID.dropna().tolist()
        elligible_movies = elligible_movies[~elligible_movies[id_type].isin(movies_with_nan_genders_ID)]
        print(len(elligible_movies))

    return elligible_movies

/tmp/ipykernel_17532/3953538505.py:3: DtypeWarning: Columns (1,3,6,9,10,11,12) have mixed types. Specify dtype option on import or set low_memory=False.
  character_metadata = pd.read_csv(data_folder+'characters_data.csv')


In [294]:
elligible_movies = select_elligible_movies_for_main_char_analysis(movie_metadata)
df = elligible_movies.sample(n=20)
df.head(20)

53542
39263


UnboundLocalError: local variable 'id_type' referenced before assignment

In [280]:
elligible_movies.head(122)

,wikipedia_ID,freebase_ID,name,release_date,box_office_revenue,runtime,genre,plot_summary,IMDB_ID
429,24708749.0,/m/04yg13l,ironclad,2011,NaN,121.0,"['Action', 'Historical fiction', 'Adventure']",The film opens with a prologue describing how ...,tt1233301
991,31972917.0,/m/0gvrx5s,creature,2011,NaN,93.0,['Horror'],"A group of teenagers; Oscar and Karen, Beth an...",tt1686018
1183,28722503.0,/m/0cz8mkh,final destination 5,2011,157887643.0,95.0,"['Thriller', 'Horror', 'Teen', 'Supernatural']","Sam Lawton, is on his way to a company retreat...",tt1622979
1247,34513262.0,/m/0j24_bp,killing season,2013,NaN,NaN,"['Thriller', 'Action']","['Two veterans of the Bosnian War, one America...",tt1480295
1287,19163173.0,/m/06ztvyx,kung fu panda 2,2011,NaN,92.0,"['LGBT', 'Wuxia', 'Adventure', 'Computer Anima...",['Po and his friends fight to stop a peacock v...,tt1302011
...,...,...,...,...,...,...,...,...,...
28343,27123043.0,/m/0bs3vgr,butter,2011,105018.0,90.0,['Comedy film'],Destiny is a 10-year-old orphan girl in Iowa ...,tt1349451
28523,28236076.0,/m/0cnx4y_,sanctum,2011,108609310.0,108.0,"['Action/Adventure', 'Thriller', 'Action', 'Dr...","Seventeen-year-old Joshua ""Josh"" McGuire , exp...",tt0881320
28648,25619875.0,/m/09v71cj,the beaver,2011,6370816.0,91.0,"['Black comedy', 'Drama', 'Comedy']","Walter Black is a depressed CEO of Jerry Co.,...",tt1321860
28708,33509716.0,/m/0bq8tmw,21 jump street,2012,201585328.0,109.0,"['Crime Fiction', 'Comedy film', 'Action']",Morton Schmidt and Greg Jenko are former cla...,tt1232829


In [ ]:
def select_elligible_movies_for_main_char_analysis(movie_metadata,ID:str):
    # Removing movies without summaries
    elligible_movies = movie_metadata[movie_metadata['plot_summary'].notna()]

    # Removing movies with characters having name listed as nan
    movies_with_nan_characters_wiki_ID = pd.Series(character_metadata[character_metadata['character_name'].isna()]['wikipedia_ID'].unique())
    movies_with_nan_characters_wiki_ID = movies_with_nan_characters_wiki_ID.dropna().tolist()
    movies_with_nan_characters_IMDB_ID = pd.Series(character_metadata[character_metadata['character_name'].isna()]['IMDB_ID'].unique())
    movies_with_nan_characters_IMDB_ID = movies_with_nan_characters_IMDB_ID.dropna().tolist()
    elligible_movies = elligible_movies[~elligible_movies['wikipedia_ID'].isin(movies_with_nan_characters_wiki_ID)]
    elligible_movies = elligible_movies[~elligible_movies['IMDB_ID'].isin(movies_with_nan_characters_IMDB_ID)]

    # Removing movies with characters having gender listed as nan
    movies_with_nan_gender_wiki_ID = pd.Series(character_metadata[character_metadata['actor_gender'].isna()]['wikipedia_ID'].unique())
    movies_with_nan_gender_wiki_ID = movies_with_nan_gender_wiki_ID.dropna().tolist()
    movies_with_nan_gender_IMDB_ID = pd.Series(character_metadata[character_metadata['actor_gender'].isna()]['IMDB_ID'].unique())
    movies_with_nan_gender_IMDB_ID = movies_with_nan_gender_IMDB_ID.dropna().tolist()
    elligible_movies = elligible_movies[~elligible_movies['wikipedia_ID'].isin(movies_with_nan_gender_wiki_ID)]
    elligible_movies = elligible_movies[~elligible_movies['IMDB_ID'].isin(movies_with_nan_gender_IMDB_ID)]

    # Removing movies with 


Optimized version of Aveek's code

In [138]:
tagger = SequenceTagger.load('ner')

def extract_main_characters(summary: str, nb_sentences=5):

    # Extracting and tagging first nb_sentences from summary
    sentences = sent_tokenize(summary)
    tagged_sentences = [Sentence(sent) for sent in sentences[:nb_sentences]]
    tagger.predict(tagged_sentences)

    # Extracting all names from the tagged sentences
    entities = [entity for sent in tagged_sentences for entity in sent.to_dict(tag_type='ner')['entities']]
    names = [entity['text'] for entity in entities if entity['labels'][0]['value'] == 'PER']

    # Removing punctuation
    names = [name.translate(str.maketrans('', '', string.punctuation)) for name in names]

    names_numbered = Counter(names).most_common()

    characters = defaultdict(int)

    for name, count in names_numbered:
        found = False
        standardized_name = name.lower()

        # Adding up number of counts if over 50% match
        for existing_name in characters:
            if fuzz.ratio(standardized_name, existing_name) > 50:
                characters[existing_name] += count
                found = True
                break

        # Adding name to character list if unique
        if not found:
            characters[standardized_name] += count

    # Converting from dictionary to ordered list
    ordered_characters = sorted(characters.items(), key=lambda x: x[1], reverse=True)
    
    ordered_characters = Counter(characters).most_common()
    main_characters = [name for name, count in ordered_characters[:3]]
    
    return main_characters

2023-11-16 22:01:44,609 SequenceTagger predicts: Dictionary with 20 tags: <unk>, O, S-ORG, S-MISC, B-PER, E-PER, S-LOC, B-ORG, E-ORG, I-PER, S-PER, B-MISC, I-MISC, E-MISC, I-ORG, B-LOC, E-LOC, I-LOC, <START>, <STOP>


In [277]:
def find_main_characters_genders(movie_row, characters_df):
    IMDB_ID_character_list = characters_df.loc[characters_df['IMDB_ID'] == movie_row['IMDB_ID']]
    wikipedia_ID_character_list = characters_df.loc[characters_df['wikipedia_ID'] == movie_row['wikipedia_ID']]
    selected_character_metadata = pd.concat([IMDB_ID_character_list,wikipedia_ID_character_list],ignore_index=True)

    genders = []
    for name in movie_row['main characters']:
        confidence = 0
        if selected_character_metadata['character_name'].any():
            closest_character, confidence, score = process.extractOne(name, selected_character_metadata['character_name'])
            
        if confidence > 50:
            gender = selected_character_metadata.loc[selected_character_metadata['character_name'] == closest_character, 'actor_gender'].values[0]
            genders.append(gender)

    return genders


In [278]:
df['main characters'] = df['plot_summary'].apply(extract_main_characters, nb_sentences=5)
df['main character genders'] = df.apply(find_main_characters_genders, characters_df=character_metadata, axis=1)
df.head(20)

,wikipedia_ID,freebase_ID,name,release_date,box_office_revenue,runtime,genre,plot_summary,IMDB_ID,main characters,main character genders
8420,27995292.0,/m/0ch5m5g,a little bit of heaven,2011,1296937.0,107.0,"['Romantic comedy', 'Romantic drama', 'Romance...","Marley Corbett, a carefree woman with a promis...",tt1440161,"[marley, julian goldstein]","[F, M]"
4438,33336349.0,/m/0h4y6y7,the collection,2012,NaN,NaN,"['Thriller', 'Action', 'Horror']",['A man who escapes from the vicious grips of ...,tt1748227,"[elena, arkin, ryan rodriguez]","[F, M, M]"
26917,30093517.0,/m/0g4tknw,detention,2011,NaN,93.0,"['Horror', 'Comedy', 'Teen']","The film opens with Taylor Fisher, the most po...",tt1701990,"[taylor fisher, riley jones]","[M, F]"
18527,29434198.0,/m/0ds009g,midnight son,2011,NaN,88.0,"['Thriller', 'Horror', 'Romance Film', 'Drama'...","Midnight Son is the story of Jacob, a young ma...",tt1068961,"[jacob, mary]","[M, F]"
26005,35370504.0,/m/0h28mms,sinister,2012,41000000.0,NaN,"['Crime Fiction', 'Horror']",The film opens on Super 8 footage where a fami...,tt1922777,"[ellison oswalt, tracy, ashley]","[M, F, F]"
17547,32913943.0,/m/0fpmjt1,circumstance,2011,454121.0,107.0,"['LGBT', 'Drama', 'World cinema']",Atafeh is the teenage daughter of a wealthy I...,tt1684628,"[atafeh, shireen, mehran]","[F, F, M]"
4509,34569548.0,/m/0hn8j9w,v/h/s,2012,87629.0,116.0,"['Thriller', 'Cult', 'Horror', 'Erotic thriller']",Directed by Adam Wingard The film is an ant...,tt2105044,[adam wingard],[M]
1903,34480764.0,/m/06w99h3,rango,2011,245375374.0,107.0,"['Western', 'Adventure', 'Comedy-drama', 'Anim...",A pet chameleon becomes stranded in the Moja...,tt1192628,"[iguana beans, gila, bill]","[F, F, M]"
22965,30818824.0,/m/0gfh84d,lawless,2012,51316673.0,116.0,"['Crime Fiction', 'Western', 'Drama']","In 1931, the Bondurant brothers—Forrest, Howar...",tt1212450,"[forrest, rakes, jack]","[M, M, M]"
22034,25370032.0,/m/09k6ykp,losing control,2011,NaN,91.0,['Comedy'],['Unable to give her boyfriend Ben (Reid Scott...,tt1511425,"[ben, reid scott, sam]","[M, F]"


In [ ]:
df